# Importing Libraries

In [1]:
# Importing Libraries
import socket
import threading
import time
import json
import msvcrt # real time input

# Utility Functions

In [2]:
def sendServerQuery(socket, msg): # Send a message and Recieve Response Also
    sendJsonMessage(socket, msg)
    
    name, msg = recvJsonResponse(socket)

    return msg
    
def sendMessage(socket, msg, name): # name is the name of sender
    jasonObj = {"Name": name, "Message": msg}
    jasonStr = json.dumps(jasonObj)
    jasonStr = jasonStr.encode()
    socket.send(jasonStr)

def sendMessageInChatRoom(sockets, senderNumber, senderName, msg, inChatUsersNumbers): # send to all who are in chat room
    for socketNumber in inChatUsersNumbers:
        jasonObj = {"Name": senderName, "Message": msg}
        jasonStr = json.dumps(jasonObj)
        jasonStr = jasonStr.encode()
        if socketNumber != senderNumber:
            sockets[socketNumber].send(jasonStr)
            
def recvJsonResponse(socket): # For only messages. If there is no name
    res = socket.recv(1024)
    res = res.decode()
    res = json.loads(res)
    
    return res["Name"], res["Message"]

def sendJsonMessage(socket, msg): # For only messages. If there is no name
    jasonObj = {"Name": None, "Message": msg}
    jasonStr = json.dumps(jasonObj)
    jasonStr = jasonStr.encode()
    socket.send(jasonStr)
    
def askUser(sockets, userNumber, myName): # If other user wants to chat
    sendJsonMessage(sockets[userNumber], myName + ", wants to chat with you. Do you want to chat? [Y/N]\nNote:- Accepting would result in you lefting any current group chats")
    name, res = recvJsonResponse(sockets[userNumber])
    if res == "Y" or res == "y" or res == "Yes" or res == "yes":
        sendJsonMessage(sockets[userNumber],  "<--------------    Chat With : " + myName + "     -------------->")
        return True
    return False
    
    
def kernel(msg, name, usersSockets, userNames, charRoomNames, usersInEachRoom, totalChatRooms, total, socket, number, chatRoom, roomNumber, chatUser, userNumber):
    if msg == "LEAVE" or msg == "leave" or msg == "Leave":
        if chatRoom[number]:
            infomessage = "----    Successfully Exit the Chat Room " +  chatRoom[number] + "    ----"
            sendJsonMessage(socket, infomessage)
            
            chatRoom[number] = None
            roomNumber[number] = None
        
        if(chatUser[number]):
            infomessage = "----    Successfully Exit the Chat with " +  chatUser[number] + "    ----"
            sendJsonMessage(socket, infomessage)
            
            
            infomessage = "----    " + str(chatUser[userNumber[number]]) + " Exit The Chat    ----"
            sendJsonMessage(usersSockets[userNumber[number]], infomessage)
            
            chatUser[userNumber[number]] = None
            userNumber[userNumber[number]] = None
            chatUser[number] = None
            userNumber[number] = None
        
    
    elif chatRoom[number]:
        sendMessageInChatRoom(usersSockets, number, name, msg, usersInEachRoom[roomNumber[number]])
    
    elif chatUser[number]:
        sendMessage(usersSockets[userNumber[number]], msg, name)

    elif msg == "K" or msg == "k":
        sendJsonMessage(socket, "Send\n-> U for users List\n-> C for chat rooms List")
    
    elif msg == "U" or msg == "u":
        sendData = ""
        for namee in userNames:
            if namee != name:
                sendData = sendData + "->" + str(namee) + "\n"
        sendData = sendData + "--- Enter The Name of any User To Enter Chat ---"
            
        response = sendServerQuery(socket, sendData)
        
        if response in userNames:
            waitmessage = "Please Wait till " +  response + " Confirms. We'll let you know as soon as we get a response"
            sendJsonMessage(socket, waitmessage)
            
            indexOfOtherUser = userNames.index(response)
            
            chatUser[number] = response
            userNumber[number] = indexOfOtherUser
            
            tempChatRoom = chatRoom[indexOfOtherUser]
            tempRoomNumber = roomNumber[indexOfOtherUser]
            chatRoom[indexOfOtherUser] = None
            roomNumber[indexOfOtherUser] = None
            
            agree = askUser(usersSockets, indexOfOtherUser, name)
            
            if agree:
                chatUser[indexOfOtherUser] = name
                userNumber[indexOfOtherUser] = number
                
                sendOk = "\n<--------------    Chat With : " + response + "     -------------->\n****  SEND 'LEAVE' TO EXIT CHAT  ****"
                sendJsonMessage(socket, sendOk)
            else:
                chatUser[number] = None
                userNumber[number] = None
                
                chatRoom[indexOfOtherUser] = tempChatRoom
                roomNumber[indexOfOtherUser] = tempRoomNumber
        else:
            sendError = "\n*** ERROR! INVALID OPTION ***" 
            sendJsonMessage(socket, sendError)
    
    elif msg == "C" or msg == "c":
        sendData = ""
        for name in charRoomNames:
            sendData = sendData + "->" + str(name) + "\n"
        sendData = sendData + "--- Enter The Name of any Room To Enter Chat ---"
        
        response = sendServerQuery(socket, sendData)
        
        if response in charRoomNames:
            indexOfchatRoom = charRoomNames.index(response)
            
            chatRoom[number] = response
            roomNumber[number] = indexOfchatRoom
            
            usersInEachRoom[indexOfchatRoom].append(number)
            sendOk = "\n<--------------    Chat in : " + response + "     -------------->\n****  SEND 'LEAVE' TO EXIT CHAT  ****" 
            sendJsonMessage(socket, sendOk)
        else:
            sendError = "\n*** ERROR! INVALID OPTION ***" 
            sendJsonMessage(socket, sendError)
            
    else:
        sendJsonMessage(socket, "\n*TRY AGAIN*\nSend\n-> U for users List\n-> C for chat rooms List")
    return chatRoom, roomNumber, chatUser, userNumber


# Multi Threading Classes

In [3]:
class send(threading.Thread):
    def __init__(self, sockets, names, total):#, thread_name, thread_ID): 
        threading.Thread.__init__(self) 
        self.sockets = sockets
        self.names = names
        self.total = total
        
        self._running = True
    
    
    def terminate(self):
        self._running = False

    def run(self): 
        while self._running:
            if(total > 0):
                print("total : " + str(total))
                name = input("Enter message > ")
                
                for csocket in self.sockets:
                    sendJsonMessage(csocket, name)



In [4]:
class receive(threading.Thread): 
    def __init__(self, usersSockets, userNames, charRoomNames, usersInEachRoom, totalChatRooms, chatRoom, roomNumber, chatUser, userNumber, total):
        threading.Thread.__init__(self)
        self.usersSockets = usersSockets # All sockets List
        self.userNames = userNames # All users names
        self.charRoomNames = charRoomNames # All chat rooms names
        self.usersInEachRoom = usersInEachRoom # List of list of users in a room
        self.totalChatRooms = totalChatRooms # total number of chat rooms
        self.total = total # total number of clients => 
        
        self.socket = usersSockets[total - 1]
        self.name = userNames[total - 1]
        self.number = total - 1
        
        self.chatRoom = chatRoom # user is in any room
        self.roomNumber = roomNumber
        self.chatUser = chatUser # user is chatting with any user
        self.userNumber = userNumber
        
        self.chatRoom, self.roomNumber, self.chatUser, self.userNumber
        
        self._running = True
    
    def terminate(self):
        self._running = False

    def run(self): 
        while self._running:
            #time.sleep(1)
            name, msg =  recvJsonResponse(self.socket)
            
            
            print(self.name + " : " + msg)
            
            kernel(msg, self.name, self.usersSockets, self.userNames, self.charRoomNames, self.usersInEachRoom, self.totalChatRooms, self.total, self.socket, self.number, self.chatRoom, self.roomNumber, self.chatUser, self.userNumber)
            
            
            

In [5]:
class chatRoom(threading.Thread): 
    def __init__(self, sockets, names, total, usersInRoom):#, thread_name, thread_ID): 
        threading.Thread.__init__(self) 
        self.socket = socket
        self.names = names
        self.total = total
        self.usersInRoom = usersInRoom
        
        self._running = True
        
        
    def terminate(self):
        self._running = False

    def run(self): 
        while self._running:
            #time.sleep(3)
            msg =  recvJsonResponse(self.socket)
            
            if msg == "C" or msg == "c":
                sendJsonMessage(self.socket, "Send\n-> Room to connect to a room\n-> U to connect to a user")
            
            
            print(self.userName + " : " + msg)



# Main Code

#### Defining some common variables

In [6]:
#setting up chat rooms
totalChatRooms = 3
charRoomNames = ['Alpha', 'Beta', 'Charlie']
threadChatRooms = []

usersRoomA = []
usersRoomB = []
usersRoomC = []

usersInEachRoom = [usersRoomA, usersRoomB, usersRoomC]

chatRoom = [] # User X in Y chat room : where chatRoom[X] = Y (Y is room name)
roomNumber = [] # User X in Y chat room number : where roomNumber[X] = Y (Y is room number)
chatUser = [] # User X chating with user Y : where chatUser[X] = Y (Y is user name)
userNumber = [] # User X chating with user number Y : where userNumber[X] = Y (Y is user number)


# User Info
usersSockets = [] # all users sockets
userNames = [] # all users names
total = 0 # total number of users connected

#### Chat Rooms Threads

In [7]:
# newThreadChatRoomA = chatRoom(usersSockets, userNames, total, usersRoomA)
# newThreadChatRoomA.start()
# newThreadChatRoomB = chatRoom(usersSockets, userNames, total, usersRoomB)
# newThreadChatRoomB.start()
# newThreadChatRoomC = chatRoom(usersSockets, userNames, total, usersRoomC)
# newThreadChatRoomC.start()

#### Socket Creating (listining for users requests)

In [ ]:
#Listening for socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind((socket.gethostname(), 12345))

threadsReceive = []

newThreadSend = send(usersSockets, userNames, total)
newThreadSend.start()


print("...Waiting for clients...")

while True:
    s.listen(5)
    clientsocket, address = s.accept()
    print("New connection : " + str(clientsocket))
    
    sendJsonMessage(clientsocket, "<> Send '' K '' to connect for chat <>")
    
    name, username = recvJsonResponse(clientsocket)
    usersSockets.append(clientsocket)
    userNames.append(username)
    
    chatRoom.append(None)
    roomNumber.append(None)
    chatUser.append(None)
    userNumber.append(None)

    
    total = total + 1
    
    newThreadReceive = receive(usersSockets, userNames, charRoomNames, usersInEachRoom, totalChatRooms, chatRoom, roomNumber, chatUser, userNumber, total)
    newThreadReceive.start() 
    threadsReceive.append(threadsReceive)
    
    
newThreadSend.join()

for thread in threadsReceive:
    thread.join()


...Waiting for clients...
New connection : <socket.socket fd=1180, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.10.5', 12345), raddr=('192.168.10.5', 58247)>
total : 1
New connection : <socket.socket fd=1216, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.10.5', 12345), raddr=('192.168.10.5', 58281)>
